# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install --upgrade tensorflow keras Pillow

Requirement already up-to-date: tensorflow in ./Deepleaning/lib/python3.8/site-packages (2.4.1)
Requirement already up-to-date: keras in ./Deepleaning/lib/python3.8/site-packages (2.4.3)
Requirement already up-to-date: Pillow in ./Deepleaning/lib/python3.8/site-packages (8.2.0)


In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.4.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./128,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./128)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[128, 128, 3]))

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 100s 396ms/step - loss: 0.7330 - accuracy: 0.5689 - val_loss: 0.5881 - val_accuracy: 0.6915
Epoch 2/25
250/250 [==============================] - 113s 451ms/step - loss: 0.5818 - accuracy: 0.6888 - val_loss: 0.6105 - val_accuracy: 0.6755
Epoch 3/25
250/250 [==============================] - 118s 473ms/step - loss: 0.5614 - accuracy: 0.7223 - val_loss: 0.4956 - val_accuracy: 0.7565
Epoch 4/25
250/250 [==============================] - 119s 476ms/step - loss: 0.5078 - accuracy: 0.7401 - val_loss: 0.5098 - val_accuracy: 0.7525
Epoch 5/25
250/250 [==============================] - 119s 477ms/step - loss: 0.4862 - accuracy: 0.7729 - val_loss: 0.4747 - val_accuracy: 0.7725
Epoch 6/25
250/250 [==============================] - 119s 476ms/step - loss: 0.4703 - accuracy: 0.7826 - val_loss: 0.4833 - val_accuracy: 0.7720
Epoch 7/25
250/250 [==============================] - 119s 476ms/step - loss: 0.4488 - accuracy: 0.7901 - val_loss: 0.4573 -

## Part 4 - Making a single prediction

In [15]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

ValueError: in user code:

    /home/ilus/DeepLearning/building-cnn/Deepleaning/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /home/ilus/DeepLearning/building-cnn/Deepleaning/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ilus/DeepLearning/building-cnn/Deepleaning/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ilus/DeepLearning/building-cnn/Deepleaning/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ilus/DeepLearning/building-cnn/Deepleaning/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.

In [ ]:
print(prediction)

## Fim